In [2]:
import os
import praw
import json
import openai

In [3]:
reddit = praw.Reddit(
    client_id=os.getenv('REDDIT_CLIENT_ID'),
    client_secret=os.getenv('REDDIT_CLIENT_SECRET'),
    user_agent='ios:python.finetune.reddit:v0.1(finetune with reddit data)'
)

In [35]:
data = []
topic = 'Steam'

for submission in reddit.subreddit(topic).top(limit=500, time_filter='year'):
    submission.comments.replace_more(limit=0)
    data.append({
        "messages":[
            {"role":"system", "content":"You are a hard-core gamer that plays every game released"},
            {"role":"user", "content":submission.title},
            {'role':"assistant", "content":submission.comments[0].body}
        ]
    })

with open('../Data/reddit_data.jsonl', 'w') as f:
    for item in data:
        f.write(json.dumps(item) + '\n')

In [3]:
client = openai.OpenAI()

In [42]:
file = client.files.create(
    file=open('../Data/reddit_data.jsonl', 'rb'),
    purpose='fine-tune'
)

In [43]:
client.fine_tuning.jobs.create(
    training_file=file.id,
    model='gpt-3.5-turbo-1106'
)

FineTuningJob(id='ftjob-Trnsxcz3f0BMB7jRHc1OKWKt', created_at=1707519642, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-8XZOQeqwpwaHNycMN6jve2hs', result_files=[], status='validating_files', trained_tokens=None, training_file='file-ahFIHvgfuEWzx5WEd8aYveHK', validation_file=None)

In [15]:
finetune_id = client.fine_tuning.jobs.list(limit=1).data[0].id

In [17]:
model_name = client.fine_tuning.jobs.retrieve(finetune_id).fine_tuned_model

In [23]:
finetune_completion = client.chat.completions.create(
    model=model_name,
    messages=[
        {'role':'user', 'content':'Can you suggest an interesting game what was released in 2023?'}
    ]
)
print(finetune_completion.choices[0].message.content)

Time travel. Wow


In [25]:
finetune_completion = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role":"system", "content":"You are a hard-core gamer that plays every game released"},
        {'role':'user', 'content':'Can you suggest an interesting game what was released in 2023?'}
    ]
)
print(finetune_completion.choices[0].message.content)

This future was as bleak as you all predicted
